# Day 17: Pyroclastic Flow

Looks kind of like tetris. https://adventofcode.com/2022/day/17

Oddly-shaped rocks appear out of nowhere in mid-air. Jets of hot gas push them to the left and right as they fall. The elephants are testing me. How tall will the tower be, after the 2022nd rock falls?

In [1]:
example = True
total_rocks = 2022

<hr>

In [2]:
import numpy as np

In [3]:
if example:
    puzzle = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

else:
    puzzle = '''>>>><<<<>><<<>><<>><<<<>><<<<><<<><<<<>>>><<<>><<<>><<<<>><<><<<>>><<<>>><<>>><<>><<<>>><><>><<<>><><<>>><<<<><<<<>><<<<><<>>>><<>>>><<<<><<<<>><>><>>><<<<>><>>>><<<<>><<<><<<<>>>><<><>>>><<<>>><<<<>><<<<><>>><<<>>><<<<>>><<<>><<>>>><<>>><><<<>>>><<<<>>>><>>>><<<<>>><<>><<<<>><<>>>><<<>><<<>>>><>>>><<>>>><>><>>>><<<<>>><<<<>><<<<>>><<<<><<<>>>><<>>>><<<><<<<><<<<>><<<<>>>><>><<<<>>><<>><<<<>>><><<<><<>><<<<>><<>><<<<><>>><<>>><<><<<<>><<<<>><<>><><<<<>><>>><<<<>>>><>><<<<>>><<<><><<<>><<><>>>><<>>>><>><<>>><<<>>><<<><<>>><<><<>>>><<>>>><<<>>>><<<<>>><>>><>><>>>><<><>>><<<<><<>>><<<<>>>><<<>>><<>><><<>>>><><<<><<<<>>>><<><<<>>><<<>>><<>>>><<<<>><<<<>><<><>>><<<<>>><<<<>>>><<<<>><<<<>>><<<<>>>><<<<>><>>><<<><<><<<><<<<>><>>>><<<<>>>><<>><>>>><<<>>>><><<<<>>><<><<<>><<>>>><<><<<>><<>>><<<<>>>><<<<>><>><>><<><<>>>><<<><>><<>>>><<<>>><>>>><<<>>><<<<>>>><<<>>><>><<><>>>><>>><>>><><<>>>><>>><<>><<<><<>><<>>><>>>><<><>>><<<<>><>>><>>><>><<<<>>>><<<<>>><<<>><<<<>>><>>>><<<>>>><<>><<>>>><>><<>><<<<><<<<>>><<>>><<>>>><<>>><<<<>>>><>>>><<<>>><<<>>><<<<><<<>>><>><<<><<<<>>><<<>>>><>>><>><<>>>><><>>><>>>><<>>>><<<>>><<>><<><<<<>>><<<>>>><>><><<<>><><<<<><<<<>>><<<>>>><<<<>>><<>>>><>>><<<<>><<>>>><>>><>><>>>><<<<>>>><<<<>>>><<<<>><<<<>><<<>>>><>>>><<<>><<<>><<<<>>>><>>><<<>>><<<<><<<<><><>>>><<<>>>><<<><<>><><<>><<<>>>><<<<><>>><<<<>>>><>>>><<>>>><<<<>>>><><<>>>><<>><<<<><<<<>><<>>>><<<>>>><><>>>><<<>><><<<><<>>><<>>>><<<>><><>><<<<>>>><<><>>><><>>>><<<<>>>><<<>>><<<>>><<><>><<<>>><<>>><<<<><>><<<>>><>><>>><<<<>><<<>>>><<<>>><<>>><>><<>>><<<<>><>>><<<<>>>><<<<><<><>><<<<>>>><<<<><>>><<<<>>><<><><>>>><<>>>><<<<>><<<<>>>><<><>>><<<<>>>><<<><<>>><<>>><<<>>>><<<>>>><<>>>><<<>>><><<<>><><<<<><<>><<<<><<<><<<>>><<<>>>><>><<<<>>>><<>>>><<>>>><<<>>>><>>>><<<>>><>><>>><>><><>>>><<><<>><<<>>>><<<><<<>>>><<<<>><<<><>>>><<<><<><>><<<><<><<<<>><>>><<<><<>>><<<<>><><<<>>><<<<>><<>><<>>>><>><<>><><<<<>><<<<>><<<<>>>><><<<>>><<>><>>><>><>>>><<<<><<>>>><>><>><<<>><<<><>><<<><>>><<<><<<<>><<<>>><<<<><>><<<<>>>><<><>>><<><<<>>>><<>>>><>>><<<<>><><<>>><<>><<<<>><<<<>>><<><<<>>><<<>><>><<<>>>><<>>><<<<>><>>>><>>><<<>><<<<>>>><>>>><<>><>><<<><<><<<<><<<<>>><<<>>><<<><><<<>><<<<>>>><<<><<<<>><<>>><<<<>><>>>><<<>><<>>>><<<<><<<<><<><<<>>>><<<>>>><<<>>><>>><<<<>>>><>>>><<<><<<><<>><<><<<<>>>><<<>>>><<<<>><<<>>><<<>>><><<<<><<>>><<><<<><<<<>><<>><>><<<>>><<>>>><<<<>>><<<<>>>><><>><<<<>>><<<<>>>><<<>>>><<>><<<<>>>><<>><<<<>>><<<<>>>><<>>><><<<<>><<><<<>>>><<>><<>><<<<>>>><<<><<<>>><<<<>><<<<>><<<>>>><<>>>><>>><<<<><<>>><<>>>><>>><<<<>>>><<>>><<<>>><<<<>><<><<<>>>><<><<<>><<><<<<>>><<<<>><<<>>>><><<<<>>>><<<<><<<><<>><<<<>><>>>><>><<<<><<<>><<<>>>><<>>><<<>>>><<<><<<<>><<<>>><<<<>>><<<><<>>>><<<<>>>><>>>><<>>>><<<><><<<<>>>><>>>><>>><><<>>><<>>>><<<>>><<>>>><<<>><<>><<<<>>>><<>>><<>>><<><>>>><>>>><<<<><<>>>><<<<>><<<<>>>><<>><>><>><<<>><<<><>><<>>>><<><<<>>>><<>>>><><<<><<<>><<<><<<<>>>><<<>><<>><><<<<>><<<<>>>><<<>>><<><<<<>>>><<>><><<<><><<<<>>><<<<>>>><>><<>>>><<>>>><<<<>>><<<<>>>><<<>><<<>><<<<>>><<<<><<>><<<><>>>><>><<<<>>><>>>><>>>><>>>><<>><<<<>><<<<>><><<<><>>>><<<>><>>>><<<<>>>><<<>><>>>><<<<><<<<>>><<<<>>><>>><<>>><>>><>>><<<>>><<>><<<>><>>>><<<<><<<><<<<>>>><<>><<<<>><<><<<<>>><<<<>><<<<>><<<><<<<>><<>><>>><<><<>><<>><<<>>>><<<<><>>>><>>><<<>>><<>><<<<>><<<<>>>><<>>><<>><<>>><>><>>>><<><<<>>>><<>>>><>>>><<><<<><>><<<<>>><<<><<>><<<<>>><<<>>><<<><<<<><<<<><<<<>><><<>>><>>>><>><<<>>>><><>><<<><<<<>><<>>><<><<><<<<><<>><<><<<>>>><<<><><<<<><<<><<<<>>><<<<><>><<<<>>>><<<>>>><<<>>>><>>><<><<>>><<<>><<<><<<><<<<><>><><><<<>>>><<><<>>><><>>><<><<<>><<>>>><<<<>>>><<<>><<<>>><><<<<>><<>>><<<>><>><<>><<>>>><<><<<>>>><>><<<<><<><<<>>>><<<<>>><<<<><><<<>><<<><<<>>><<<<><<<>><<<>><<>>>><>>><<<>>>><<<><><>><<<<>>><><<<><<><<>>><<<<><<<<>><<<<>>>><<<<>>><<<<><<><<>><><<>>>><<<>>>><<>>>><<<>>>><<>>><>><<<>>><<<<><>>><<>>><>><<>><<><<<><<<>><><><>>>><<<>><<<<><<<>>><<<>>>><>>><<>><<><<<<>>><<>>><<>>><<<<>>><><<<>>><<<>>>><<><>>><<><>>>><<<<>>><<<>><<<<>><<<<>><>>><<<<>>>><>>>><<>>>><<><<>><><>>><<<<>>><<><<<>>>><<>><<<<>>><<><<><<><><<<<><>><>>><<<>><<><<>><<<<>>>><<<>><<<>><<<<>>>><<>>>><<>>><<<>>><<<><<<>>>><><><<<>>>><>>>><<<>>><>><<<><<>><<<<>>>><<<>>>><<<<>>>><>>>><<<<>><>>>><<<<><>><<<<>><<<>><<<>><<<<>>><>>><>>><>>><<<>>>><<<<>><<<>><<><>><<>><>>><>>>><<>><<<<>><<>>>><>>>><<>>>><<<<><>>>><<<><<><<<<><<>>><<<>>><>><<><<<>>>><<<>>><<>>><><<><>>><<<<>>>><>>><<>><><<<<>>>><>>>><<<><<<<>><<>>>><<<<><<<<>>><<<><<<>>>><>>><<<<>>>><<<<>>>><<><<<<>><<<<><>>>><>>><><<>><<<<><<<<>><>>>><<<<><>>><<<>>>><<<>>><<<<>>><<><<<<>>><<<>><<<<>>><<<>><<>>>><<<>><<<<>>><<<>>>><<<<>><<>>>><<<>><<<<><<<<>>><<<<>>>><><<<>>>><><<<<>><<>>>><>>>><<>>>><<><<><>>><>><<<<><<<><<<><<>>><<<>><<<<>>><<<>><<>>><>><>>>><<<>>>><<>>>><<<>>>><<<<>>>><<<>>><<<<>><>><><>>><<<<>><<<>><<<>>>><<<<>><>>>><<>>><<<<><>><<<>>><<<>>>><<<><<<<>>>><<<><>>>><<<<>><>>><<>><><<<<>><<<<>><<<>>><<<>>>><<<<>>><>>>><<>><><<><<>><<<><<<>><<<><><<<<><<>><<<<>>>><<<<>>>><<>>>><<<<>>><<<<>>>><<<<>>>><>>><<<>><<<>><<><<<><<<>><<<<><<>>>><>>><<<>>><><<>>><<<>><>>>><<<>><<>><<<<>><<<<>>>><<<<>>><<>>>><>>>><<<<>>><>><<>>><<>><<<<>>><<>><<<<>>><<>>><>>><<<>>><<><<><<>><><>>><<><>><>>><>><<<<>>>><<<>><>>><<<>><<<<>><<>>>><<><><<<<><>><>><<>><<<>><<<<>>>><<<<>>>><><<<>>><<<<>>>><<<<><<<<>>>><<<>>><<<<>>><<<>>><<<><<<>><>><>>><<<><>><>>>><>>>><<<>>><<><<<<>>><<>>>><<<><<<<>><<>><>>><<<><>>><<<>><<<<>><<<<><<<<><<<<>><><<>><<<<>>>><<<>>><<<<>>><<<><<<><<<<>><<<><>>>><<>><<<<>><<<<>>><>>>><<<<>>>><<<<>><>>><>>>><<<>>>><<><>><<><>>>><>>>><<>>><<<>>><<>>>><<>><>>><<<>><<<>>>><>>><<<><>>>><<><>>><<<<>><>>>><<<>><<<><<<><<>>><<<>>>><<<>>><<<<>>><<>>>><>>>><<>>>><<<<><<<>>><<<<><<>><>>><<<><<>>><>>><<<<>>>><<<><<<><>><<<>>><><<<>><<<<>>><<<<>>>><<<>>><<>>>><<>>>><>><<<>>>><>>>><<>>><<><<<><<<<><<<<><>>>><>>><<<>>>><<<<>>>><<<<>>><<>>>><>><>><<>><>>>><<<><<<<><<>>><<><<>>><<<<><<<>>><<<<>>><<>><<><<<>>>><>>>><>>><>>>><<<>>>><<<><<>><><>><>>><<<>>>><<>><<<><<<<><<>>>><<><>>>><<><>>>><<><><<<<>>><><<>><<<><>><>><<>>><<<>>>><><<>>>><<<<><<<<>>><>>>><<<<>><<<<>>>><<<>>><<>>>><><<<<>><<<<><<<<><<>><><<<>>><<<<>>><<<>><<<<>>>><<<>>><<<<>>>><>>><<<>><<<<>>><<<<><<<>><<<>>><<<>>>><<>>>><><<<>>>><<<>>><<>>>><><<>>><<<<>>>><><<><<<<>>>><<>><<<<>><<>><<<<><<<>><<<<>>><>>>><<<>><<<>>>><<<>><>><<<>><>><>><<<><<<<>>>><>>><<<<><>>><>>>><>>><<<<>>><<<>>>><>>>><<<<>>>><>>><>>><>>>><<>>><<<<>><<<>>>><<>>><>><<<<>><<<>><<<<>>>><>>><<<<>>><<>>>><>>>><<<>>>><<<>><<<<>><<<>><<<>><<<<>><<<<>><>><<<<>>><<<>><<<><<<<>><<<<>>>><>><>>><<<<><<<>>>><<>>><<<<>><<<>>>><<>>><>>>><<>>><<<<><<>>><<<<>>>><>>>><><<<>>><<<>><<>>>><<<>>><<<<>><<<>>>><><>>>><<<<>><<<<>>>><<<>>>><>>>><<<<>>>><<<<>>><<>>><<<><<<>><<<<><<<>>>><>>>><>>>><<<<>>>><<<><<>><<<<><>>><<>>><<<>>>><<<<>>>><<>>><>><<>><<><>>>><<<><<><>>>><>>><>>><<<><<<>><>>><<<<>>><<<>>><<>><<>><<<>>>><<>>>><>><<><<<<>>><<<>><<<<>><>><<<>>><>><><<>>>><<<><<><<>>>><<><<<><><<<<>><<>><<<<>>>><>>><<<>><<<<>><<<<>><<<<>>><<<<>>>><<<<>>><>>><<><<>>>><<>>><>>>><<<><<>><>>>><<<><<><<<<>>>><>>>><><<<><>>>><<<>>>><<<>>>><<<<><><<<<>>><<<<>>>><<>>><<<<>>>><><<<<>>><<>>><<<<>><><<<<>><>><>>>><>>>><><<>>>><<<<>><<<<>>><><<<>>>><<><>><>>><<>>>><>>><<><<<>>><><><>><>><<<><<><>>><<>>>><<>>><>><><<<<>>>><><>>><<<<>>>><<><<><<<<>><<><>>>><<><>>><<>><>><<<><<<<>>>><<<><>>><<<>>>><<<<>><>>><<>><<<>><<<<>>>><>>><<<<>>><<<>><<<>>><<<>>><<>>><<<><<<<>>>><<<>>><<<<>>><>><<><<<><<<>>>><<<<>>>><<<<>>><<<><<<><<<><<<>>>><<><<><<>><><<<<>>><<<>>>><<<>><><<<><<<<>>><<>><>>><<><>>><<<><>>><>>><<<<><<>>><>>>><>>>><<<<>>>><>>><<><<<>><<<>><>>><>><<<>>><<<>>><<<>>>><<<<><<><<<>>>><><<<<>><<><<><<<<>>>><<<>><<<>>>><<><<<>>><<><<<<>>>><<<>>><>>><>>><<<>>><<<<>>>><<<<>>><><<>><<>><<<<>>>><>><<<>>>><<><<>><<>><>><<<>>>><<><<<>><<<<><<<<><<<<><<<><>><>><>>><<<<><<<<>>>><<><<>><<<<>>><<<><>>>><<<>>>><<><>>>><<>>>><<<>>>><>><<><<>>><<<>>><<><<<>>><>>>><<<><<><<<><<>>><><<<>>><<><>><><<>>>><<<<>>>><<<><<<<><<<<>>><>><<>><<<<>><>>><<<>><>>>><<><><<<<>>>><<>>><<>>>><<>>>><<<>>><<>><<>><<<>><>>><<><<<><<<><<<<>>>><<>>><<<><<>><<<>><><<<>>>><<>><<<>>><<<<>>>><<<><<<<>>>><><<<<><<>>><<>><<<>>>><<>><><><<<<>>>><>>>><<>>><<<><<<>><<<>>>><<<>>>><>>><<<<><>>>><<>><>>>><<<<><<<<>><<<>><><<<>>>><<<<>>>><>><<<>>>><>>>><<<>><>>>><<>><<<>>>><>><<>>>><<<<>>><>>><>>><<<>>><>>>><<>>>><<>>>><<<>>><<<<>>><<<><<<><>>><<<<><<<>>>><><<<<>>><<><<<<><<><<<>>><<<<>>><<<<><<>>><<<>>>><<<>><>><>>>><<>>><<<><<<>>>><<><<>><<<<>>><<>><<<><<>>>><<>>><>><><<<><<<>>>><>>>><<>>><<<<><<<>>><<<<>>>><<<<><><>><>>><<>><>>><<><>>><<><>>>><<<<>>><<>><<>>>><<<>>>><<<>><<<<>><>>>><<>>>><><<>>><><<><>>>><<<<>><<<>><>>>><>>><>>><>>>><><>><<<>><>><<>>>><<>>>><>>><<>><><<>><>><<<><<<>>>><<<<>>><<<><><<<<>>>><>>><<<<>><<<>>><>>>><<>><<<>>>><<><<<<>>>><<><<>>>><>>><>><<>>>><<><<>>>><<<<><<<<>>>><>>><<<<>>>><>>>><<<>>>><<<<>>><<>><<<>>><<<<>>><<>>><<<<><>>><<<<>>>><<<><<<>>>><<<>><><>>><>>>><>><<<<>><>>>><<>>><<<<>><>>><<>>><<<>><<<<>>>><<<>>>><<>>>><<><<<>>>><<<>><<>>><<<<>><<>>>><>>><<>>>><<<<><<>>>><<<<><<>>>><<<<>>><>>>><>>>><<<>>>><<<>>><<<<>>><<<><<<<>><<><>>><>><><>>><<<<>>>><<>><<>><<<<>>><<<><>><<<><<<<>>>><<<<>>>><<<>><<<>><<<>><<<<>>><<<<>>>><<<<>>>><<<<>><<>>><>>>><<<<><<<>><<<>><><<>>>><<<<>><<>><<>>>><><<><>>><<<><<<><<>><>>><<<<><<<>><<<>>><><<<>><<>>><<<>>><<<<>>><<><<<<>>>><<<>>><>>>><<>>>><>><>><<><<<<>>><<<<>><<>><<>>>><<<<>>>><<>>>><<><>>>><<>>>><<<>><<>>>><<>>><<<>><<>>>><<<>><>><<>><<<<>>>><<<><<>><<<<><><<<><<>>><<<>>>><<<<>>><<>>><>><<><>>>><>>>><>>>><<<<>>>><<<>>><<>>>><<<<>>>><>><<<<>>><<<<>>>><>><<><>>>><<<<><>><<>><<>>>><<<<>>><<<>>><<<>>><>>>><>><<<<>><<<>>>><>>><<<>>><<<>><>>>><>>><<>><<<<><>><<<><<<>><<<>><<<<>>><<<<>>><<>>><<<<>>><<<<>><>>><<>><<>>>><>>><<<<>>><<<<><<<>>>><<<>>>><>>>><<>><<>><<>>><>>><>>><<<>>>><<<><>>><>><<<>><<<<>><<<<>>>><<<>>><>><<<<>>>><<<<>>>><<>>>><<>>><>><<<>>><<<<>>>><>>><<<<><<>><<>><<<<>>><<<<>>>><>>>><><<<<><><<<<>>>><>>><<><<<>><<>><<>>>><>>>><<<>><>><<><<<>>>><>>><><<>>>><<<<>>><<<<>>><>>>><<<<>><<<>>><<<><<<>><<><<<>>>><<<<>>><<<>>><<<>>><>>><>><<>>>><<>>><<>>>><<>>>><><<<>>><>><<>><<>>>><<<><<>><<<<>><<><>><<<<><>>><<>><>>'''

Rock can move left, right, or down if it is not blocked by the walls or by resting rock.

In [4]:
def blocked_left(tower):
    '''blocked if any part of a moving rock is in leftmost column or up against resting rock'''
    for row in tower:
        if row[0] == 1:
            return True
        else:
            for i in range(len(row)):
                if row[i] == 1 and row[i-1] == 2:
                    return True
    return False


def moveleft(tower):
    '''shift moving rock to left'''
    if not blocked_left(tower):
        for rownum in range(len(tower)):
            for position in range(len(tower[rownum])):
                if tower[rownum, position] == 1 and tower[rownum, position-1] == 0:
                    tower[rownum, position-1] = 1
                    tower[rownum, position] = 0
    return tower
                

def blocked_right(tower):
    '''blocked if any part of a moving rock is in rightmost column or up against resting rock'''
    for row in tower:
        if row[len(row)-1] == 1:
            return True
        else:
            for i in range(len(row)):
                if row[i] == 1 and row[i+1] == 2:
                    return True
    return False


def moveright(tower):
    '''shift moving rock to right'''
    if not blocked_right(tower):
        for rownum in range(len(tower)):
            for position in range(len(tower[rownum])-1, -1, -1):
                if tower[rownum, position] == 1 and tower[rownum, position+1] == 0:
                    tower[rownum, position+1] = 1
                    tower[rownum, position] = 0
    return tower
                

def blocked_below(tower):
    '''blocked if any part of a moving rock is directly above resting rock'''
    for rownum in range(len(tower)-1):
        for position in range(len(tower[rownum])):
            if tower[rownum, position] == 1 and tower[rownum + 1, position] == 2:
                return True
    return False


def movedown(tower):
    '''shift moving rock down'''
    for rownum in range(len(tower)-2, -1, -1):
        for position in range(len(tower[rownum])):
            if tower[rownum, position] == 1 and tower[rownum+1, position] == 0:
                tower[rownum+1, position] = 1
                tower[rownum, position] = 0
    return tower
                

def solidify(tower):
    '''mark all moving rock as resting rock'''
    for i in range(len(tower)-1):
        for j in range(7):
            if tower[i, j] == 1:
                tower[i, j] = 2
    return tower


def settle(tower):
    '''rock falls if possible, or comes to rest if blocked'''
    if blocked_below(tower):
        rest = True
        tower = solidify(tower)
    else:
        rest = False
        tower = movedown(tower)
    return rest, tower
        

In [5]:
rocks = []
rocks.append(np.array([[0, 0, 1, 1, 1, 1, 0]]))
rocks.append(np.array([[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0]]))
rocks.append(np.array([[0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0]]))
rocks.append(np.array([[0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]))
rocks.append(np.array([[0, 0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0]]))

spacer = np.array([[0 for i in range(7)] for j in range(3)])


In [6]:
tower = np.array([[2, 2, 2, 2, 2, 2, 2]])

jetseq = puzzle

rocknum = 1

while rocknum <= total_rocks:
    rocktype = (rocknum - 1) % 5
    rock = rocks[rocktype]
    tower = np.vstack((rock, spacer, tower))
    rest = False
    
    while not rest:        
        jet = jetseq[0]
        if len(jetseq) == 1:
            jetseq = puzzle
        else:
            jetseq = jetseq[1:]

        if jet == '<':
            tower = moveleft(tower)
        else:
            tower = moveright(tower)        
        
        rest, tower = settle(tower)

    while len(tower) > 0:
        if sum(tower[0,:]) > 0:
            break
        else:
            tower = np.delete(tower, 0, 0)
            
    
            
    rocknum += 1

In [7]:
print('Tower height:', len(tower) - 1)

Tower height: 3068


It would be much more efficient to work with only the section of the tower that the rock is falling through. But the time to code that is much longer than the time to just let it run, so I decided not to obsess.



## Part 2

Now the elephants want me to simulate one trillion rocks. The current approach won't scale, even if I did restrict the part of the tower I'm working with. The rock shapes are a repeating pattern, and so are the jets. There's probably a repeating pattern in the pile, too.

In [8]:
len(puzzle)

40

In [9]:
for row in range(len(tower) - 255, len(tower)):
    s = ''
    for symbol in tower[row]:
        s += '.@#'[symbol]
    print(s)

.##...#
..#...#
..#.###
..#..#.
..#.###
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..
..####.
....###
.....#.
.#####.
.#..#..
.#..#..
.####.#
.####.#
###.###
.#####.
.###...
.###...
.#.#...
.#.#.#.
.######
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
....#..
...###.
#...#..
#####..
#.#....
#.#....
####...
..#####
...#.##
..####.
.##....
.##...#
..#...#
..#.###
..#..#.
..#.###
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..
..####.
....###
.....#.
.#####.
.#..#..
.#..#..
.####.#
.####.#
###.###
.#####.
.###...
.###...
.#.#...
.#.#.#.
.######
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
....#..
...###.
#...#..
#####..
#.#....
#.#....
####...
..#####
...#.##
..####.
.##....
.##...#
..#...#
..#.###
..#..#.
..#.###
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..


I found, by staring at and coloring this stack in Excel, that after the first 15 rocks in the example, which stack a height of 25 rows, a repeating pattern in the tower appears. The repeating pattern is 53 layers tall and contains 35 rocks.

The jet sequence in the puzzle input has length 10091, compared to the example sequence of length 17. I need a better way to look for a pattern.

What if we tracked tower height vs. rocks dropped? for 1000 rocks

In [29]:
import pandas as pd

In [30]:
total_rocks = 1000
rocks_dropped = [0]
tower_height = [0]


tower = np.array([[2, 2, 2, 2, 2, 2, 2]])
jetseq = puzzle
rocknum = 1

while rocknum <= total_rocks:
    rocktype = (rocknum - 1) % 5
    rock = rocks[rocktype]
    tower = np.vstack((rock, spacer, tower))
    rest = False
    
    while not rest:        
        jet = jetseq[0]
        if len(jetseq) == 1:
            jetseq = puzzle
        else:
            jetseq = jetseq[1:]

        if jet == '<':
            tower = moveleft(tower)
        else:
            tower = moveright(tower)        
        
        rest, tower = settle(tower)

    while len(tower) > 0:
        if sum(tower[0,:]) > 0:
            break
        else:
            tower = np.delete(tower, 0, 0)
            
    rocks_dropped.append(rocknum)
    tower_height.append(len(tower) - 1)
            
    rocknum += 1

dic = {'rocks': rocks_dropped, 'height': tower_height}
data = pd.DataFrame(dic)
data

,rocks,height
0,0,0
1,1,1
2,2,4
3,3,6
4,4,7
...,...,...
996,996,1510
997,997,1513
998,998,1516
999,999,1520


In [31]:
len(tower) - 1


1520

In [32]:
for row in range(len(tower)):
    s = ''
    for symbol in tower[row]:
        s += '.@#'[symbol]
    print(s)

....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..
..####.
....###
.....#.
.#####.
.#..#..
.#..#..
.####.#
.####.#
###.###
.#####.
.###...
.###...
.#.#...
.#.#.#.
.######
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
....#..
...###.
#...#..
#####..
#.#....
#.#....
####...
..#####
...#.##
..####.
.##....
.##...#
..#...#
..#.###
..#..#.
..#.###
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..
..####.
....###
.....#.
.#####.
.#..#..
.#..#..
.####.#
.####.#
###.###
.#####.
.###...
.###...
.#.#...
.#.#.#.
.######
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
....#..
...###.
#...#..
#####..
#.#....
#.#....
####...
..#####
...#.##
..####.
.##....
.##...#
..#...#
..#.###
..#..#.
..#.###
.#####.
....#..
....#..
....#..
....#..
.##.#..
.##.#..
..###..
...#...
..###..
...#...
..####.
..###..
..###..
..####.
....###
.....#.
.#####.
.#..#..
.#..#..


 You know what? I don't care any more.